In [44]:
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler

n = 16
q = 32768
t = 256
sigma = 8
delta = q//t

S.<x> = PolynomialRing(ZZ)
R.<zeta> = S.quotient(S.ideal(x**(n) + 1, q))

SampleSmallPoly = lambda :  R(DiscreteGaussianDistributionPolynomialSampler(ZZ['x'], n, sigma)())

def round_to_nearest(x, delta):
    # a // b = floor(a/b), where RHS is rational division
    # (a + (b//2))//b ~ floor(a/b + 1/2) = round_to_nearest_int(a/b)
    return (x + delta//2) // delta

def poly_round(poly, delta):
    return R(lift(poly).map_coefficients(lambda c: round_to_nearest(c, delta)))


In [69]:
def keygen():
    return SampleSmallPoly()

def enc(m, s):
    a = R.random_element()
    e = SampleSmallPoly()
    print("a = ", a)
    print("e = ", e)
    return (a, a*s + e + delta*m)

def dec(ct, s):
    (a, b) = ct
    partial_dec = b - a * s
    return R(partial_dec.lift().map_coefficients(lambda c: round_to_nearest(c, delta)))

def hom_sum(ct0, ct1):
    (a0, b0) = ct0
    (a1, b1) = ct1
    return (a0+a1, b0+b1)

def hom_prod_and_dec(ct0, ct1, s):
    (u0, v0) = ct0
    (u1, v1) = ct1
    num = v0*v1 - s * (u0*v1+v0*u1) + s*s*u0*u1
    print("num = ", num)
    print("num.parent() = ",num.parent())
    print("delta^2 = ", delta**2)
    print("q =", q)
    return poly_round(num, delta**2)
    

In [70]:
s = keygen()
m0 = 3
m1 = 2
ct0 = enc(m0, s)
ct1 = enc(m1, s)
hom_prod_and_dec(ct0, ct1, s)

num =  -12*zeta^30 - 144*zeta^29 - 23*zeta^28 - 104*zeta^27 - 332*zeta^26 + 98*zeta^25 - 403*zeta^24 - 43*zeta^23 - 130*zeta^22 - 550*zeta^21 + 58*zeta^20 - 579*zeta^19 + 128*zeta^18 - 664*zeta^17 - 157*zeta^16 - 4491*zeta^15 + 2402*zeta^14 + 1113*zeta^13 - 6249*zeta^12 + 7876*zeta^11 - 4702*zeta^10 + 728*zeta^9 - 1073*zeta^8 - 5620*zeta^7 - 4084*zeta^6 - 4024*zeta^5 - 1221*zeta^4 - 6345*zeta^3 - 478*zeta^2 - 3591*zeta + 95616
num.parent() =  Quotient of Univariate Polynomial Ring in x over Integer Ring by the ideal (x^16 + 1, 32768)
delta^2 =  16384
q = 32768


6